# Processing of particle tracks 
### <font color='gray'>coded to work with experiments and simulation</font>

## _Import of python libraries used within the code_

### <font color ='gray'> load both code cells  </font>  

In [1]:
#HI HELLO
import numpy as np # fast array library
import pandas as pd # data frames library
# plot display type. use this to keep graphs within the notebook
import matplotlib


import matplotlib.pyplot as plt # plot library

import subprocess # these 2 lines below load OS functionality
import sys
import json

from IPython.display import display, HTML # screen on display in HTML library 

import sympy as sp # symbolic calculation library functions
from sympy import var # symbolic calculation library variables

import mpmath as mp # double (or higher) floating point precision library (not used)

import scipy.constants as ct # import mathematical constants
import scipy.stats as st
import scipy.special as spec

import ipywidgets as widgets # import 

from scipy.spatial import Voronoi, voronoi_plot_2d # import Voronoi functions

import scipy.optimize as optimize # optimazation library (not used)
from collections import Counter # (not used)
import time # timing functions (for computation time)

from scipy import signal # noise filtering functions

global iscaled 
iscaled = 0

global LX, LY
LX = 1000 -250
LY = 650-50

# system size for a given (npart, rho) configuration
def L(npart, rho):
    l = np.sqrt(npart*2*np.sqrt(3.)/rho)
    return l

# print progress function
# more elegant (python 3 alternative): print('\rhello', end='',flush=True)
def printp(string):
    sys.stdout.write('\r'+ str(string))
    sys.stdout.flush()
    
#from __future__ import print_function

# Text format for figures configuration (lines below)
from matplotlib import rc
rc('text', usetex=True)
plt.rcParams['font.size'] = 20
# special plotting functions
from matplotlib.collections import PolyCollection
import matplotlib.colors as mcolors

def set_up_graf(idf, LX, LY):
    global ax, fig_system, ss

    fig_system = plt.figure(idf, figsize=(6.5,6.5*LY/LX))
    ss=(72./fig_system.dpi)**2 # particle size
    ax = fig_system.add_subplot(1, 1, 1)
    ax.set_ylim([0,LY])
    ax.set_xlim([0,LX])
    ax.set_xlabel(r'$x/\sigma$',fontsize = 24)
    ax.set_ylabel(r'$y/\sigma$', fontsize = 24)


In [2]:
%matplotlib --list

Available matplotlib backends: ['tk', 'gtk', 'gtk3', 'wx', 'qt4', 'qt5', 'qt', 'osx', 'nbagg', 'notebook', 'agg', 'svg', 'pdf', 'ps', 'inline', 'ipympl', 'widget']


In [2]:
# matplotlib widgets interface
%matplotlib widget 
# tk interface
#%matplotlib tk 

## _Read tracking data_ 

We explain the action of each of these functions below:

__get_info(hash_prefix) :__ Reads and prints info file with  _hash prefix_ hash code. See output table in <font color='red' >RUN CELL 0</font> below to grasp the info structure

__simple_pickle_read(hash_prefix) :__
This 'simple_pickle_read' takes a pkl.xz file _hash prefix_ hash code, and which contains xy positions and tracks (no velocities) and stores in 'tabla' pandas data frame.
   
  * input: hash_prefix
  * output: global variable _Ntracks_ (number of tracks) 

In [3]:
def pdisplay(info):
    display(HTML(info.to_html()))

# This function reads pickle binary file (.pkl) with trajectories
def get_info(series_directory, hash_prefix):
    global info
    nombre = '/data/Datos/chiral/'+ series_directory +'/info/' + hash_prefix + '.txt'
    with open(nombre) as f:
        jsonstr = json.load(f)
    info = pd.json_normalize(jsonstr)
    pdisplay(info.T)
    return info
    
def simple_pickle_read(series_directory, hash_prefix):
    global Nframes, Ntracks
    # Read table in pickle format
    nombre = '/data/Datos/chiral/' + series_directory + '/tracks/900fps/' + hash_prefix \
       + '.pkl.xz'
    tabla = pd.read_pickle(nombre, compression='infer')
    tabla = tabla.reset_index(drop=True) # por si los indices salen desordenados
    Nframes = np.max(tabla.frame)+1
    Ntracks = np.max(tabla.track)+1
    tabla = tabla.sort_values(by = ['frame', 'track']).reset_index(drop=True)
    return tabla

## _Reduce global table - functions_

__reset_track_indexes(tabla0):__ 
Eliminates 'lonely' tracks (particles tracked for just 1 frame), re-indexes so that no track indexes are empty
* input: _tabla0_ original data frame (of tracks)
* output: RETURNS clean _tabla_ data frame, without empty tracks; AND original _tabla0_ with empty tracks still included
* output structure: _tabla0_, _tabla_

__short_drop:(ishort, tabla):__ 
Eliminates tracks detected for n frames or less, re-indexes so that no track indexes are empty

* Input: _ishort_ (threshold length of tracks: shorter tracks are not kept), _tabla_ (original tracks data frame)

* Output: RETURNS _tabla_\__short_ (frame), that is the input table _tabla_ without tracks shorter than _ishort_


In [4]:
def reset_track_indexes(tabla0):
    global Ntracks, Nframes
    """ This function takes a dataframe in which some trajectory indexes
        are missing (maybe due to having deleted short trajectories) and
        resets indexes so that we can loop over the tracks with 'range' """
    # 'real_number_of_tracks' should be <=  than 'current_last_particle_index'
    tabla = tabla0.copy()
    Ntracks = len(set(tabla.track))
    original_indexes = np.sort(list(set(tabla.track)))
    unsort_indexes = original_indexes
    fixed_indexes = np.arange(0, Ntracks, step=1, dtype=int)
    if  (original_indexes == fixed_indexes).all()==False: # fix only if there are empty tracks
    # With these two lists we create a dictionary and map old values to new ones
        n_empty = np.max(tabla.track) - Ntracks
        replacement_dict = dict(zip(original_indexes, fixed_indexes))
        tabla.track = tabla.track.map(dict(zip(original_indexes, fixed_indexes)))
        print('no. of empty track indexes discarded: ', n_empty, '\n')
    else:
        print('nothing to fix\n')
    Ntracks = np.max(tabla.track)+1
    Nframes = np.max(tabla.frame)+1
    return tabla0, tabla

# INPUT
# ishort: number of minimum frames in a track (eliminates tracks under ishort time length)
# tabla: pandas Data Frame to shorten
# OUTPUT
def short_drop(ishort, tabla):
    global shorts_list, Ntracks
    shorts_list =[]
    Ntracks = np.max(tabla.track)+1
    for i in range(Ntracks):
        t1 = track(i,tabla,False)
        if  len(t1) < ishort+1:
            shorts_list.append(i)
            len0 = len(tabla)
            tabla = tabla.drop(t1['index'])
            texto = 'dropped track no. '+str(i)+'; data table length decreased in '+str(len(tabla)-len0)
            printp(texto)
    print('\n')
    tabla_short = tabla.sort_values(by = ['frame', 'track']).reset_index(drop=True)
    Nshorts = np.max(tabla_short.track)+1
    printp('Dropped out ' + str(Ntracks-Nshorts)+' short tracks out of ' + str(Ntracks))
    # the line above is necessary so that eliminates index voids and shuffling after short drop
    print('\n')
    return tabla_short


## _Get tracks and states from global table - functions_

__track(t_id, tabla, dropit) :__  builds track for one particle from the appropriate chunk of the source table, with only the lines for particle _t_\__id_ 

* Input: _t_\__id_ indice de track, _tabla_ source frame fuente (contains tracks), _dropit_ boolean; if =True then erases old index column (tracks do not always begin in frame 0); most of the time you just want _dropit_=True

* Output: RETURNS a chunk of source table _tabla_ with only the lines for particle _t_\__id_

__all\___ __tracks(tabla, dropit):__ repeats the process in _track_ function for all existing particles

* Input: _tabla_ (frame) data source, _dropit_ boolean variable set True to erase original table original line number

* Output: RETURNS _track_ array of frames each wiith one particle track; builds _tr_\__lengths_ array of tracks lengths


__state(it, tabla):__ gets one instantaneous state frome source frame _tabla_, at frame _it_

* Input: _it_ frame no. to get the state from; _tabla_ source frame data

* Output: RETURNS st frame chunk from _tabla_ source frame

__all__\___states(tabla):__ Builds all instantaneous states from the movie

* Input: _tabla_ source frame data

* Output: RETURNS array of frame chunks from _tabla_ source frame. each chunk being an instantaneous state

In [5]:
#####  INDIVIDUAL TRACKS ##############
# build a 't_id' indivitual track 
def trackf(t_id, tabla, dropit):
    t1 = tabla.loc[tabla.track == t_id].reset_index(drop=dropit)
    return t1

# build individual tracks from all kept tracks
# OUTPUT
# tr_lengths[i]: length of track no.  'i'. The total no. of tracks is stored in 'Ntracks'
def all_tracks(tabla,dropit):
    # length of track
    global tr_lengths
    tr_lengths = np.empty(Ntracks,dtype=int)
    tracks = [[] for i in range(Ntracks)]
    for i in range(Ntracks):
        tracks[i] = trackf(i,tabla,dropit)
        tr_lengths[i] = int(len(tracks[i]))
    return tracks

#####  INSTANTANEOUS STATES  ##############
# BUILD INSTANTANEOUS STATES OF THE SYSTEM
def state(it, tabla):
    st = tabla.loc[tabla.frame == it].reset_index(drop=True)
    # reset row index 
    #(otherwise keeps chunked index of the original table)
    return st

# build instantaneous states over all frames
def all_states(tabla):
    sts = [[] for i in range(Nframes)]
    for i in range(Nframes):
        sts[i] = state(i,tabla)
    return sts


## _Low-Pass filters_ 

__butter_lowpass(step,fps, arr) :__ Applies Butterworth low-pass filter to _arr_ array, with averaging width _step_, for a movie with _fps_ frame rate; _fr_ is the fraction of the maximum frequency that is allowed to pass

* Input: _fr_, _step_, _fps_, _arr_; as described above
* Output: RETURNS an array of the same size as _arr_

__cheby1_lowpass(step,fps, arr) :__ Applies Chebyshev type I low-pass filter to _arr_ array, with averaging width _step_, for a movie with _fps_ frame rate

* Input: _step_, _fps_, _arr_; as described above
* Output: RETURNS an array of the same size as _arr_



In [6]:
def butter_lowpass(fr,step,fps, arr):
    
    #N, Wn = signal.buttord(1./(1.*fps/step),1./fps, 1/step, fps*0.5)
    N, Wn = signal.buttord(fr/step, fr, 1/step, fps*0.5 ,0.5/fps)
    b, a = signal.butter(N, Wn,'low')
    yy = signal.filtfilt(b, a, np.squeeze(arr), padtype=None)
    return yy

def filter_tracks_butter(fr,step,fps,tabla):
    tabla_unfiltered = tabla.copy()
    for i in range(Ntracks):
        printp('Filtering positions for track no. ' + str(i+1) + ' of ' + str(Ntracks))
        xbb = np.array(tabla.loc[tabla['track']==i,'x'])
        xb = butter_lowpass(fr,step,info.fps[0], xbb)
        tabla.loc[tabla['track']==i,'x'] = xb
        ybb = tabla.loc[tabla['track']==i,'y']
        yb = butter_lowpass(fr,step,info.fps[0], ybb)
        tabla.loc[tabla['track']==i,'y'] = yb
    print('\n')
    return tabla_unfiltered, tabla

#def cheby1_lowpass(step,fps,arr):
#    N, Wn = signal.cheb1ord(1./(1.*fps/step),1./fps, step, fps*0.5)
#    b, a = signal.cheby1(N, 1./fps, Wn, 'low')
#    y = signal.filtfilt(b, a, arr)
#    return y


## _Get velocities (differences) and accelerations (differences of differences) from individual tracks_

__vels(tabla) :__ Ouputs velocities from tracks table; by decomposing into individual trajectories first

* Input: _tabla_ frame data source; str01: name of _x_ coordinate inpuy variable; str02: same for _y_ coordinate

* Output: RETURNS _tabla_ with 2 new columns (_tabla['vx']_, _tabla['vy']_)

In [7]:
def diffs(str01,str02,str1,str2,tabla):
    global last_id
    tabla[str1] = np.zeros(len(tabla)) # creates new columns for differences
    tabla[str2] = np.zeros(len(tabla))
    last_id = np.zeros(Ntracks, dtype=int) # creates array for storing strack last index
    for k in range(Ntracks):
        printp('diffs: ' + str(k+1)+ ' of ' + str(Ntracks) + ' tracks')
        t1 = track(k,tabla,False)
        tabla.loc[t1['index'][:-1],str1] = np.diff(t1[str01]) #store differences in new columns
        tabla.loc[t1['index'][:-1],str2] = np.diff(t1[str02])
        last_id[k] = tabla.loc[tabla['track']==k].index.values[-1:]
    tabla = tabla.drop(last_id) # gets rid of last point in track (has no defined diff)
    print('\n')
    return tabla
    
def angle_diffs(tabla):
    global last_id
    tabla['w'] = np.zeros(len(tabla)) # creates spin column
    for k in range(Ntracks):
        printp('diffs: ' + str(k+1)+ ' of ' + str(Ntracks) + ' tracks')
        t1 = track(k,tabla,False)
        tabla.loc[t1['index'][:-1],'w'] = np.diff(t1['theta']) 
        last_id[k] = tabla.loc[tabla['track']==k].index.values[-1:]
    tabla = tabla.drop(last_id) # gets rid of last point in track (has no defined diff)
    print('\n')
    return tabla


## Bring data to physical units

We take as coordinate origin the mid-point of the max and min X and Y positions ever tracked within the region of interest (ROI) during the experiment.

We have 2 options for lengh unit: 
* milimeters 
* Ball diameter

The time scale unit is seconds.

Only after this step position differences in the original data table are re-scaled as real velocities

In [8]:
# PHYSICAL SCALES AND ORIGIN
# Usage: re_pos_scale(0) for 1 mm as length unit; re_pos_scale(1) for ball diameter (sigma) 
# length unit

def set_origin(shiftx, shifty, tabla):
    global ishifted
    if ishifted == 0:
        tabla_not_shifted = tabla.copy()
        tabla['x'] -= shiftx
        tabla['y'] -= shifty
        shifted = 1
    return tabla_not_shifted, tabla


def scale(l_unit, t_unit, tabla):
    global iscaled
    tabla_not_scaled = tabla.copy()
    if iscaled == 0:  
        inv_l_unit = 1./l_unit
        print(inv_l_unit)
        if np.any(tabla.columns=='vx') or np.any(tabla.columns=='vy'):
            print(inv_l_unit)
            tabla[['x','y']] *= inv_l_unit
            tabla[['vx', 'vy']] *= (inv_l_unit * t_unit)
            if np.any(tabla.columns=='ax') or np.any(tabla.columns=='ay'):
                tabla[['ax', 'ay']] *=  (inv_l_unit * t_unit**2)
        else:
            tabla[['x','y']] *= inv_l_unit
        iscaled = 1
    else:
        tabla[['x','y']] *= 1
        iscaled = 0
    return tabla_not_scaled, tabla


## Processing of blades angles

The process has the following steps:
 
* Eliminating blades with angles: $\theta_i >2 \pi$
* Storing angle list length
* Interpolating missing angles for those lists with angles length < no. of blades

In [9]:
# save max and/or min angles and re-scale angle unit
circ_unit = 14 # NUMBER OF BLADES IN PARTICLES

def save_rescale_angle(imin, tabla):
    ln = len(tabla)
    maxs = np.array([tabla.extremos[i][0] for i in range(ln)]) # keeps angle maxima
    maxs = circ_unit*(maxs/360)
    datos['maxt'] = maxs # transforms angle into blade units
    if imin != 0:
        mins = np.array([datos.extremos[i][1] for i in range(ln)])   
        mins = circ_unit*(mins/360)
        datos['mint'] = mins


# deletes blades beyond the forteenth blade (the last one in this case) + 1/2 blade
def filter_spurious_angles(imin,tabla):
    for i in range(len(tabla)):
        tabla.at[i,'maxt'] = np.array(list(filter(lambda x: x<(circ_unit+0.5), tabla.maxt[i])))
    if imin != 0:
        for i in range(len(tabla)):
            tabla.at[i,'mint'] = np.array(list(filter(lambda x: x<(circ_unit+0.5), tabla.mint[i])))
            

def filter_outside_blades(tabla):
    for i in range(len(tabla)):       
        try:
            #np.where(tabla.maxt[i]>14)[0][0]
            lista =  np.where(tabla.maxt[i]>14)[0]
            if len(tabla.maxt[i]) > 15:
                print("DANGER! DANGER! DANGER!\n")
            if len(tabla.maxt[i]) > 14: 
                for k in lista:
                    tabla.at[i,'maxt'] = np.delete(tabla.at[i,'maxt'],k) 
                    # deletes outside blades (+1/2) if list length > 14
            else:
                for k in lista:
                    tabla.at[i,'maxt'][k] = tabla.at[i,'maxt'][k] - 14
                    a = tabla.maxt[i][k]
                    tabla.at[i,'maxt'] = np.delete(tabla.maxt[i], k) 
                    # substract nblades to outside blades if list length < 14
                    tabla.at[i,'maxt'] = np.insert(tabla.maxt[i], 0, a)
                    # place it at the beginning of the list
        except:
            pass


# Deletes one excess element in maxima arrays
# based on  closest to 1 difference (between first and last pair)
def last_del15(tabla,larr):
    if len(np.where(larr==15)[0])>0:
        for i in np.where(larr==15)[0]:
            arr = tabla.maxt[i]
            if (np.abs(arr[1]-arr[0]-1) < np.abs(arr[14]-arr[13] -1)):
                tabla.at[i,'maxt'] = np.delete(arr,-1)
            else:
                tabla.at[i,'maxt'] = np.delete(arr,0)


# Stores in a list the angles lists lengths; of either maxt or mint (not simultaneously)
def angles_list_lengths(arr):
    la = []
    for i in arr:
        la.append(len(i))
    la = np.array(la).astype(int)
    return la


# insert_missing: 
# detects missing blades, linearly interpolates them
def insert_missing(tabla, larr):
# PROCEDIMIENTO PARA INTERPOLAR listas de maximos con menos de 14 maximos
    while len(np.where(larr==13)[0]) > 0:
        interpolated_indexes = np.zeros(len(np.where(larr==13)[0]))
        for i in np.where(larr < 14)[0]:
            diff_arr = tabla.maxt[i][1:] - tabla.maxt[i][:-1]
            imax = np.where(diff_arr == np.max(diff_arr))[0][0] # maximum index
            interpolated_indexes = np.append(interpolated_indexes, imax)
            tabla.at[i,'maxt'] = np.insert(tabla.maxt[i], imax+1, 0.5 * (tabla.maxt[i][imax] + tabla.maxt[i][imax+1]))
        larr = angles_list_lengths(datos.maxt)
    print('done\n')



## return_next: 
### finds angle location of a reference blade (j0) in next frame jt+1, within track no. itr
__input__ : _itr_ (track no.), _jt_ (reference frame no.), _j0_ (reference blade)

In [20]:
nblades = circ_unit

def return_next(itr,jt,j0):
    
    list_its = [j0, j0-1, j0+1]
    
    theta0 = tracks[itr].maxt[jt][j0]
    # by default, angle difference is with itself in the next frame
    diff0 = tracks[itr].maxt[jt+1][j0] - theta0 # ldiff0, rdiff0
    
    if (j0==0):
        ldiff = tracks[itr].maxt[jt+1][nblades-1] - theta0 - circ_unit 
        list_its[1] = nblades-1
    else:
        ldiff = tracks[itr].maxt[jt+1][j0-1] - theta0 
        
    if (j0==nblades-1):
        rdiff = tracks[itr].maxt[jt+1][0] - theta0 + circ_unit
        list_its[2] = 0
    else:
        rdiff = tracks[itr].maxt[jt+1][j0+1] - theta0
    
    list_diffs = [diff0, ldiff, rdiff]

    inm = np.where(np.abs(list_diffs) == np.min(np.abs(list_diffs)))[0][0] 
    
    in0 = list_its[inm]
    diffnext0 = list_diffs[inm]

    return in0, diffnext0


def shift_back(itr,jt,n0):
    arr_copy = np.zeros(nblades)
    for i in range(n0,nblades):
        arr_copy[i-n0] = tracks[itr].maxt[jt][i]
    for i in range(0,n0):
        arr_copy[(nblades-1)-(n0-1)+i] = tracks[itr].maxt[jt][i]
    return arr_copy

## <font color='RED'>RUN CELL 0</font>
### - Import pkl.xz tracks table
### - create all individual tracks arrays
### - analyze track length histogram, decide minimum track length

In [132]:
# COMPLETE SET OF READING INSTRUCTIONS

series_directory = 'rho_0_025'
hash_prefix = '4bf1a08dbd71379766e543b9532df560'

print('data reading and basic processing\n')

info = get_info(series_directory, hash_prefix)

datos = simple_pickle_read(series_directory, hash_prefix)
datos_orig, datos = reset_track_indexes(datos)

datos = datos[['frame','track','x','y','extremos']]

save_rescale_angle(0, datos)

del datos['extremos'] # remove original clumpsy angles list structure
#datos.columns = ['frame','track','x','y','maxs','mins'];

# imprime (formateada) cabecera de tabla de datos
pdisplay(datos.head())


print('\n min for positions:\n', np.min(datos[['x','y']]), '\n')
print('max. for positions:\n', np.max(datos[['x','y']]), '\n')


print('filtering spurious angles \n')
filter_spurious_angles(0,datos)

la = angles_list_lengths(datos.maxt)

#plt.figure(figsize=(8,8/ct.golden))
#plt.hist(la,rwidth=0.96, bins=[11,12,13,14,15,16,17],color='r',density=True,align='left')


print('filtering close outside blades\n')
filter_outside_blades(datos)

la = angles_list_lengths(datos.maxt)

plt.figure(figsize=(8,8/ct.golden))
plt.hist(la,rwidth=0.96, bins=[11,12,13,14,15,16,17],color='r',density=True,align='left');


print('delete last arrays with one excess blades (last one is < (nblades+1/2 blade) )\n')
last_del15(datos,la)

la = angles_list_lengths(datos.maxt)

plt.figure(figsize=(8,8/ct.golden))
plt.hist(la,rwidth=0.96, bins=[11,12,13,14,15,16,17],color='r',density=True,align='left');

print('interpolating missing maxima\n')
insert_missing(datos, la)

la = angles_list_lengths(datos.maxt)

plt.figure(figsize=(8,8/ct.golden))
plt.hist(la,rwidth=0.96, bins=[11,12,13,14,15,16,17],color='r',density=True,align='left')


print('storing all tracks (maxima arrays corrected)')
# store tracks individually
tracks = all_tracks(datos, True)
print('14-blades max angle lists ready')

np.where(la!=14)

theta = np.zeros((Ntracks,Nframes), dtype=float )
omega = np.zeros((Ntracks,Nframes), dtype=float)

for j in range(Ntracks):
    ln = len(tracks[j])
    theta[j][0] = np.mean(tracks[j].maxt[0]- np.arange(circ_unit))
    ianext, omega[j][1] = return_next(j, 0, 0)
    for i in range(1, ln-2):
        if (i%500)==0:  printp('track no. ' + str(j+1)+ ' of ' + str (Ntracks) + ', frame ' + str(i+i0)+ ' of ' + str(Nframes-2))
        theta[j][i] = np.mean(tracks[j].maxt[i]- np.arange(-ianext ,circ_unit -ianext))
        ianext, omega[j][i+1] = return_next(j, i , ianext)
        
plt.figure()
plt.hist(omega[0],bins=200);

data reading and basic processing



,0
experiment_id,4bf1a08dbd71379766e543b9532df560
original_file,/mnt/beegfs/malopez/serieAspas/serieAspas_N25_p20_fps900_1.cine
date,2038-01-19 04:18:16
shape,"[1280, 800]"
fps,900
exposure,1109
n_frames,24981
recording_time,27.7567
camera_distance,0.95
pixel_ratio,1089


nothing to fix



,frame,track,x,y,maxt
0,1,0,814.647468,643.994447,"[0.25638748591339144, 1.2869536860665933, 2.3682712138872364, 3.1937013154004217, 4.062835056528261, 5.028594554910648, 6.051079019082025, 6.999336460333851, 8.052447664177237, 9.050799479743016, 10.054388358909737, 11.112015727378107, 12.188752872378693, 13.217099472788039, 14.256387485913393, 15.286953686066594, 16.368271213887233]"
1,1,1,833.440082,155.107584,"[0.9291104187434944, 1.9536246836072984, 2.8379517502815976, 3.8618640749791213, 4.829868900846899, 5.8952097778688755, 6.879166968006806, 7.839304154082814, 8.81314467257208, 9.774271764505135, 10.71511339493727, 11.757090877853486, 12.867908434351998, 13.867189042760337, 14.929110418743495, 15.953624683607298]"
2,1,2,712.325766,155.562226,"[0.6147619458795595, 1.644570082871637, 2.715176844332671, 3.6690788479480614, 4.594254981365086, 5.6269320741507824, 6.6077440711242, 7.573699255395264, 8.529919752721016, 9.412321670161607, 10.458673900178939, 11.46681698482907, 12.513062634275363, 13.580008551446912, 14.61476194587956, 15.644570082871637]"
3,1,3,655.528404,229.414274,"[1.1743462723760367, 2.2167188040916868, 3.065721287246481, 4.144207459744404, 5.081257868403718, 6.089945527019442, 7.055835907889318, 8.10645277292843, 9.017277355288208, 9.998599272340504, 10.964410813930959, 12.089279736663514, 13.105769418834777, 14.074670277984042, 15.174346272376036, 16.216718804091688]"
4,1,4,574.409841,238.649058,"[0.3136588246867156, 1.3748650938500868, 2.4756963405889176, 3.3074041424426417, 4.332455716432182, 5.349232655488059, 6.346616421905119, 7.2375397489886595, 8.246514800644054, 9.155109742015783, 10.117142829136327, 11.148915380696026, 12.244352813912078, 13.26873156955934, 14.313658824686716, 15.374865093850087]"



 min for positions:
 x    437.795667
y    122.745921
dtype: float64 

max. for positions:
 x    986.920536
y    690.644348
dtype: float64 

filtering spurious angles 

filtering close outside blades



...

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

interpolating missing maxima

done



Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

storing all tracks (maxima arrays corrected)
14-blades max angle lists ready
track no. 25 of 25, frame 24500 of 24978

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

(array([], dtype=int64),)

In [71]:
return_next(20, 3,1 )

(1, 0.06661782583917342)

In [54]:
return_next(0, 5, 1)

(1, 0.22411921315816263)

In [72]:
ianext = np.zeros((Ntracks,Nframes), dtype=int)
    
for j in range(Ntracks):
    ln = len(tracks[j])
    ianext[j, 0], diff_next = return_next(j, 0, 0)
    for i in range(1, ln-2):
        if (i%500)==0:  printp('track no. ' + str(j+1)+ ' of ' + str (Ntracks) + ', frame ' + str(i)+ ' of ' + str(Nframes-2))
        ianext[j, i], diff_next = return_next(j, i , ianext[j, i-1])

track no. 25of25, frame 24500 of 24978

In [109]:
theta = np.zeros((Ntracks,Nframes), dtype=float )
omega = np.zeros((Ntracks,Nframes), dtype=float)

for j in range(Ntracks):
    ln = len(tracks[j])
    theta[j][0] = np.mean(tracks[j].maxt[0]- np.arange(circ_unit))
    ianext, omega[j][1] = return_next(j, 0, 0)
    for i in range(1, ln-2):
        if (i%500)==0:  printp('track no. ' + str(j+1)+ ' of ' + str (Ntracks) + ', frame ' + str(i+i0)+ ' of ' + str(Nframes-2))
        theta[j][i] = np.mean(tracks[j].maxt[i]- np.arange(-ianext ,circ_unit -ianext))
        ianext, omega[j][i+1] = return_next(j, i , ianext)
        

track no. 25 of 25, frame 24500 of 24978

In [133]:
plt.figure()
plt.hist(omega[20],bins=200);

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [103]:
np.arange(14)

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13])

In [94]:
return_next(0,0,0)

(0, 0.18166454084656464)

In [101]:
tracks[2].maxt[1]

array([ 0.76332978,  1.88126851,  2.82500018,  3.79074767,  4.78030227,
        5.77896073,  6.81936531,  7.79304973,  8.69888916,  9.69836879,
       10.59155193, 11.63172992, 12.74387116, 13.76317749])

In [96]:
tracks[0].maxt[0]-np.arange(-0,circ_unit-0)

array([ 0.25638749,  0.28695369,  0.36827121,  0.19370132,  0.06283506,
        0.02859455,  0.05107902, -0.00066354,  0.05244766,  0.05079948,
        0.05438836,  0.11201573,  0.18875287,  0.21709947])

In [167]:
len(np.arange(-ianext[20][4], 14-ianext[20][4]))

14

In [41]:
np.mean(tracks[9].maxt[58]- np.arange(-ianext[9][58] ,circ_unit -ianext[9][58]))

10.868228511572738

In [59]:
ianext[20][4]

1

In [52]:
tracks[20].maxt[:5]

0    [0.8420182967484177, 1.8951661871053722, 2.723...
1    [0.9150491538330071, 1.9186767460616727, 2.895...
2    [0.991162585431946, 1.9880004551589885, 2.8546...
3    [0.08122675082190867, 1.0885534329515254, 2.08...
4    [0.11676757455150977, 1.1551712587906988, 2.19...
Name: maxt, dtype: object

In [46]:
tracks[20].maxt[4]- np.arange(-ianext[20][4-1] ,circ_unit -ianext[20][4-1])

array([ 0.11676757,  0.15517126,  0.19317921,  0.01036831, -0.02218357,
        0.02040032,  0.01176516,  0.01790095,  0.0739842 ,  0.05614701,
        0.02978298,  0.07769383,  0.19897495,  0.18188973])

In [159]:
np.arange(-ianext[20][0],circ_unit)

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13])

In [140]:
np.arange(Nframes-1,dtype=int)

array([    0,     1,     2, ..., 24976, 24977, 24978])

In [138]:
len(tracks[20])

24979

In [146]:
## store tracks individually
#tracks = all_tracks(datos, True)
# Histogram of trajectory length
plt.figure(figsize=(8,8/ct.golden))
plt.title('Trajectory lengths')
# set upper height limit for histogram
track_lengths = [len(tracks[i]) for i in range(Ntracks)]
# number of bins for track length histogram
nbins = 200
#plot
plt.ylim(0,1.25*Ntracks)
fig=plt.hist( track_lengths, nbins, color='b')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [119]:
plt.figure(figsize=(8,8/ct.golden))

# Histogram of trajectory length
datos_length = len(datos.maxt)-1


# set upper height limit for histogram
edges_lengths = [len(datos.maxt[i]) for i in range(datos_length)]
# number of bins for track length histogram


fig = plt.hist( edges_lengths, bins =[11,12,13,14,15, 16],\
                color='b',density=True, align='left' ,rwidth=0.95)

s= '11 edges: ' + str("{0:.2%}".format(fig[0][0])) +\
'\n12 edges: ' + str("{0:.2%}".format(fig[0][1])) +\
'\n13 edges: ' + str("{0:.2%}".format(fig[0][2])) +\
'\n14 edges: ' + str("{0:.2%}".format(fig[0][3])) + \
'\n15 edges: ' + str("{0:.4%}".format(fig[0][4]))

plt.text(11.75, 0.7, s= s , horizontalalignment='center',fontsize=12)

plt.title('propellers no. of edges')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0.5, 1.0, 'propellers no. of edges')

In [115]:
len(edges_lengths)

624474

In [117]:
len(datos.maxt)

624475

In [109]:
plt.figure(figsize=(8,8/ct.golden))
plt.hist( angulos*(14/2)%1, bins=10,\
                color='b',density=True, align='left' ,rwidth=0.95)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

(array([1.18317491, 1.26867109, 0.91781487, 0.96526293, 0.93129826,
        0.96340536, 0.93831218, 0.9645103 , 0.93281954, 0.93477319]),
 array([3.39595906e-06, 1.00002970e-01, 2.00002543e-01, 3.00002117e-01,
        4.00001691e-01, 5.00001264e-01, 6.00000838e-01, 7.00000412e-01,
        7.99999986e-01, 8.99999559e-01, 9.99999133e-01]),
 <a list of 10 Patch objects>)

In [138]:
for i in range(len(datos)):
    datos.at[i,'maxt'] = np.array(list(filter(lambda x: x<2, datos.maxt[i])))

In [139]:
tracks = all_tracks(datos, True)

## <font color='RED'>RUN CELL 1</font>

### - obtain position differences  (vels)
### - obtain velocities differences  (accels)
### - create all (non-short) individual tracks arrays. (minimum theoretical length: 5)

In [243]:

## scale and re-position the system (left-bottom corner is (0,0))
ishifted = 0

not_shifted, datos = \
set_origin(info.ROI_center[0][0],info.ROI_center[0][1], datos) # re-position

iscaled = 0
not_scaled, datos = \
scale(info.particle_diameter_px[0], 10**3/info.fps[0], datos) # scale

#
## calculate velocities for all tracks
#datos = diffs('x','y','vx','vy', datos);
#
##calculate accelerations for all tracks
#tracks_table[ishort] = diffs('vx','vy','ax','ay', datos);
#print table head
#
#print('scaled: \n')
display(HTML( datos.head().to_html() ))
#
#
## store tracks individually
#tracks = all_tracks(datos, True)
#
## store system (instantaneous) states individually
#states = all_states(datos)


0.012658227848101266


,frame,track,x,y,theta
0,1,0,-8.278377,-4.960103,"[[17.35402463626132, 45.0, 74.47588900324574, 98.13010235415598, 127.56859202882748, 154.79887635452494, 180.0, 205.20112364547506, 232.4314079711725, 257.4711922908485, 276.7098368077569, 298.07248693585296, 326.3099324740202, 353.99099404250546, 377.35402463626133, 405.0], [6.009005957494525, 32.005383208083494, 57.9946167919165, 83.6598082540901, 109.6538240580533, 137.48955292199918, 165.06858282186246, 190.0079798014413, 215.53767779197437, 241.69924423399365, 266.18592516570965, 286.3895403340348, 315.0, 341.565051177078, 366.00900595749454, 392.0053832080835, 417.9946167919165]]"
1,1,1,-8.275366,-5.038438,"[[3.1798301198642345, 29.74488129694222, 59.03624346792648, 83.99099404250548, 107.35402463626133, 132.51044707800085, 158.19859051364818, 180.0, 206.565051177078, 235.30484646876602, 262.40535663140855, 282.5288077091515, 310.23635830927384, 338.1985905136482, 363.1798301198642, 389.7448812969422], [18.43494882292201, 45.0, 71.56505117707799, 96.00900595749452, 116.56505117707799, 141.84277341263095, 167.47119229084848, 190.61965527615513, 216.86989764584402, 244.79887635452496, 270.0, 298.07248693585296, 327.2647737278924, 356.18592516570965, 378.434948822922, 405.0]]"
2,1,2,-8.294772,-5.038365,"[[9.462322208025617, 37.568592028827496, 66.37062226934319, 90.0, 116.56505117707799, 141.34019174590992, 166.75948008481282, 188.9726266148964, 214.69515353123396, 242.10272896905238, 270.0, 288.434948822922, 317.29061004263855, 345.06858282186244, 369.4623222080256, 397.5685920288275], [25.016893478100023, 50.71059313749964, 78.69006752597979, 101.30993247402021, 126.2538377374448, 151.69924423399362, 176.18592516570965, 199.44003482817618, 225.0, 251.565051177078, 276.3401917459099, 303.6900675259798, 331.92751306414704, 360.0, 385.01689347810003, 410.71059313749964]]"
3,1,3,-8.303873,-5.026532,"[[22.380135051959574, 49.398705354995535, 79.38034472384487, 101.30993247402021, 127.56859202882748, 151.92751306414706, 180.0, 202.38013505195957, 229.3987053549955, 254.47588900324575, 277.59464336859145, 302.7352262721076, 330.6422464572087, 360.0, 382.3801350519596, 409.3987053549955], [10.619655276155134, 37.568592028827496, 63.43494882292201, 90.0, 111.25050550713325, 137.29061004263855, 162.6459753637387, 186.3401917459099, 211.60750224624888, 237.9946167919165, 263.6598082540901, 288.434948822922, 317.4895529219991, 345.9637565320735, 370.61965527615513, 397.5685920288275]]"
4,1,4,-8.316871,-5.025052,"[[17.35402463626132, 45.0, 74.47588900324574, 97.59464336859145, 124.69515353123397, 149.03624346792648, 173.6598082540901, 195.94539590092285, 222.51044707800082, 249.44395478041653, 270.0, 295.20112364547504, 324.4623222080256, 352.8749836510982, 377.35402463626133, 405.0], [6.009005957494525, 32.7352262721076, 59.03624346792648, 83.99099404250548, 109.4400348281762, 132.87890360333856, 160.3461759419467, 180.0, 206.565051177078, 232.4314079711725, 259.38034472384487, 282.5288077091515, 310.6012946450045, 340.3461759419467, 366.00900595749454, 392.7352262721076]]"


## _Plotting functions_ 

__frameit(ax):__ frames a figure by drawing the image limits and the ROI limits as well

__plt__\___track(t_id,xs,tagit,fr\_it):__ plots just one track (_t_\__id_) within _fr_\__it_ (if True) frames, with size _xs_ and prints the track no. if _tagit_ is True

__plt__\___tracks(init_id, final_id,xs,tagit,fr\_it):__ plots _init_\__id_ to _final_\__id_ tracks within _fr_\__it_ (if True) frames, with size _xs_ and prints the track no. if _tagit_ is True

In [21]:
def frameit(ax):
    ax.set_xlim(0,info['shape'][0][0])
    ax.set_ylim(0,info['shape'][0][1])
    rect = plt.Rectangle([250,50], 750, 600, alpha=1, lw=10,fill=False, edgecolor='b')
    ax.add_artist(rect)

# Plot just one track function (tagit?, frameit?)
def plt_track(t_id,xs,tagit,fr_it):
    if fr_it==True:
        fig, ax = plt.subplots(figsize=(xs,xs*info['shape'][0][1]/info['shape'][0][0]))
        px_size = 72./fig.dpi
        frameit(ax)
    else:
        fig, ax = plt.subplots()
        px_size = 72./fig.dpi
        
    plt.plot(tracks[t_id].x,tracks[t_id].y, '.', c='r', markersize=px_size,linewidth=None)
    if tagit==True:
        plt.text(np.mean(tracks[t_id].x), np.mean(tracks[t_id].y), str(t_id))
            
def plt_tracks(init_id,final_id,xs,tagit,fr_it):
    if fr_it==True:
        fig, ax = plt.subplots(figsize=(xs,xs*info['shape'][0][1]/info['shape'][0][0]))
        px_size = 72./fig.dpi
        frameit(ax)
    else:
        fig, ax = plt.subplots()
        px_size = 72./fig.dpi
    for i in range(init_id,final_id):
        plt.plot(tracks[i].x,tracks[i].y, '.', c='r', markersize=px_size,linewidth=None)
        if tagit==True:
            plt.text(np.mean(tracks[i].x), np.mean(tracks[i].y), str(i))
